# Eclipse Impact Parameter Cross-Check

Quick sanity checks on systems with observed eclipses and a merge with raw catalog parameters (period, mags, radii).

In [ ]:
import pandas as pd
from pathlib import Path

# Paths: support running from repo root or analysis/notebooks
cwd = Path.cwd().resolve()
if (cwd / "analysis" / "results").exists():
    # Running from repo root
    results_dir = cwd / "analysis" / "results"
    raw_dir = cwd / "data" / "raw"
elif (cwd.parent / "results").exists():
    # Running from analysis/notebooks
    results_dir = cwd.parent / "results"
    raw_dir = cwd.parent.parent / "data" / "raw"
else:
    raise FileNotFoundError("Cannot locate analysis/results directory from current working directory")

mcs_results_path = results_dir / "mcs_eclipse_impact_parameter_mcmc.csv"
tpc_results_path = results_dir / "tpc_eclipse_impact_parameter_mcmc.csv"
mcs_raw_path = raw_dir / "Ariel_MCS_Known_2025-07-18.csv"
tpc_raw_path = raw_dir / "Ariel_MCS_TPCs_2025-07-18.csv"

# Load
mcs = pd.read_csv(mcs_results_path)
tpc = pd.read_csv(tpc_results_path)
mcs_raw = pd.read_csv(mcs_raw_path)
tpc_raw = pd.read_csv(tpc_raw_path)

print(f"Loaded: MCS {len(mcs)} rows, TPC {len(tpc)} rows")

In [ ]:
def prepare_merge(df: pd.DataFrame, raw: pd.DataFrame) -> pd.DataFrame:
    """
    Align planet names and add key raw parameters.
    """
    add_cols = [
        "Planet Name",
        "Star Name",
        "Star Temperature [K]",
        "Star H Mag",
        "Star J Mag",
        "Planet Period [days]",
        "Eccentricity",
        "Inclination",
        "Impact Parameter",
        "Transit Depth [%]",
        "Planet Radius [Rjup]",
        "Planet Radius [Re]",
        "Planet Transit Temperature [K]",
        "Planet Eclipse Temperature [K]",
    ]
    available = [c for c in add_cols if c in raw.columns]
    merged = df.copy()
    merged["Planet"] = merged["Planet"].astype(str)
    raw_subset = raw[available].copy()
    raw_subset = raw_subset.rename(columns={"Planet Name": "Planet"})
    merged = merged.merge(raw_subset, on="Planet", how="left")
    return merged

mcs_merged = prepare_merge(mcs, mcs_raw)
tpc_merged = prepare_merge(tpc, tpc_raw)

print(f"Merged columns added: {set(mcs_merged.columns) - set(mcs.columns)}")

In [ ]:
# Flag observed/known eclipses
def add_observed_flag(df: pd.DataFrame) -> pd.DataFrame:
    flag = df.get("eclipse_observed")
    df = df.copy()
    df["observed_flag"] = flag.astype(str).str.lower().isin(["true", "1", "yes", "y", "t"])
    return df

mcs_obs = add_observed_flag(mcs_merged)
tpc_obs = add_observed_flag(tpc_merged)

print(
    f"Observed flags → MCS: {mcs_obs['observed_flag'].sum()} / {len(mcs_obs)}, "
    f"TPC: {tpc_obs['observed_flag'].sum()} / {len(tpc_obs)}"
)

In [ ]:
# Quick summaries for flagged systems
def summarize_subset(df: pd.DataFrame, label: str):
    subset = df[df["observed_flag"]]
    if subset.empty:
        print(f"{label}: no observed flags")
        return
    print(f"\n{label}: {len(subset)} observed")
    print(subset[[
        "Planet", "tier_label", "b_occ_median", "b_occ_16", "b_occ_84",
        "acceptance_fraction", "Planet Period [days]", "Eccentricity", "Inclination"
    ]].head())
    print("Tier counts:\n", subset["tier_label"].value_counts())

summarize_subset(mcs_obs, "MCS")
summarize_subset(tpc_obs, "TPC")

In [ ]:
# Optional: save enriched tables for manual inspection
out_dir = results_dir
mcs_obs.to_csv(out_dir / "mcs_eclipse_impact_parameter_with_raw.csv", index=False)
tpc_obs.to_csv(out_dir / "tpc_eclipse_impact_parameter_with_raw.csv", index=False)
print("Saved enriched CSVs with raw parameters included.")